This Jupyter Notebook implements a BERT-based similarity model using MLflow for tracking, managing, and deploying the model. It loads a pre-trained BERT model, computes sentence embeddings, and retrieves the most similar sentences from a stored corpus based on cosine similarity.

# 🔧 Import Dependencies

In [1]:
import os
import json
import shutil
import torch
import numpy as np
import pandas as pd
from tabulate import tabulate
import mlflow
import mlflow.pyfunc

from mlflow import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec, TensorSpec, ParamSchema, ParamSpec

from transformers import pipeline, AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

from nemo.collections.nlp.models.language_modeling import BERTLMModel

# ⬇️ Downloading the Bert Large Uncased Model

In [2]:
BERTLMModel.from_pretrained(model_name="bertlargeuncased", strict=False)

[NeMo I 2025-03-12 17:25:37 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.22.0/bertlargeuncased/ca4ebba9f05a8ffb79845249ca046983/bertlargeuncased.nemo.
[NeMo I 2025-03-12 17:25:37 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.22.0/bertlargeuncased/ca4ebba9f05a8ffb79845249ca046983/bertlargeuncased.nemo
[NeMo I 2025-03-12 17:25:37 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2025-03-12 17:25:46 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    data_file: /home/yzhang/data/nlp/bert/47316/hdf5/lower_case_1_seq_len_512_max_pred_80_masked_lm_prob_0.15_random_seed_12345_dupe_factor_5_shard_1472_test_split_10/books_wiki_en_corpus/training/
    max_predictions_per_seq: 80
    batch_size: 16
    shuffle: true
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
    
[NeMo W 2025-03-12 17:25:48 modelPT:617] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2025-03-12 17:25:48 modelPT:728] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 4.375e-05
        maximize: False
        weight_decay: 0.01
    )


[NeMo W 2025-03-12 17:25:48 lr_scheduler:890] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2025-03-12 17:25:49 save_restore_connector:249] Model BERTLMModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.22.0/bertlargeuncased/ca4ebba9f05a8ffb79845249ca046983/bertlargeuncased.nemo.


BERTLMModel(
  (bert_model): BertEncoder(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-

# 🏗️ Defining the BERT Similarity Model Class

In [3]:
class BERTSimilarityModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """
        Load precomputed embeddings, corpus, and the pre-trained BERT model.
        """
        # Load precomputed embeddings and corpus data
        self.embeddings_df = pd.read_csv(context.artifacts['embeddings_df'])
        self.corpus_df = pd.read_csv(context.artifacts['corpus_df'])
        
        # Load tokenizer for BERT
        self.tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
        
        # Set device to GPU if available, otherwise use CPU
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Load pre-trained BERT model
        self.bert_model = BERTLMModel.restore_from(context.artifacts['bert_model'], strict=False).to(self.device)
    
    def generate_query_embedding(self, query):
        """
        Generate BERT embeddings for the input query.
        """
        self.bert_model.eval()  # Set model to evaluation mode
        
        # Tokenize the input query and move tensors to the selected device
        encoded_input = self.tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=128)
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}
        
        # Get the model's output embedding
        with torch.no_grad():
            output = self.bert_model.bert_model(**encoded_input)
        
        # Return the [CLS] token embedding as a NumPy array
        return output[:, 0, :].cpu().numpy()
    
    def predict(self, context, model_input, params):
        """
        Compute similarity between query and precomputed embeddings,
        then return the top 5 most similar results.
        """
        # Extract the query string from model input
        query = model_input["query"][0]
        
        # Generate query embedding
        query_embedding = self.generate_query_embedding(query)
        
        # Compute cosine similarity between query and precomputed embeddings
        similarities = cosine_similarity(query_embedding, self.embeddings_df.values)
        
        # Get indices of top 5 most similar results
        top_indices = np.argsort(similarities[0])[::-1][:5]
        
        # Retrieve corresponding results from the corpus
        results = self.corpus_df.iloc[top_indices].copy()
        results.loc[:, 'Similarity'] = similarities[0][top_indices]
        
        # Return results as a dictionary
        return results.to_dict(orient="records")
    
    @classmethod
    def log_model(cls, model_name, demo_folder="demo"):
        """
        Logs the model to MLflow with appropriate artifacts and schema.
        """
        # Define input and output schema
        input_schema = Schema([ColSpec("string", "query")])
        output_schema = Schema([
            TensorSpec(np.dtype("object"), (-1,), "List of Pledges and Similarities")
        ])
        params_schema = ParamSchema([ParamSpec("show_score", "boolean", False)])
        
        # Define model signature
        signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=params_schema)
        
        # Define necessary package requirements
        requirements = ["transformers==4.47.0", "huggingface-hub==0.20.2"]
        
        # Log the model in MLflow
        mlflow.pyfunc.log_model(
            model_name,
            python_model=cls(),
            artifacts={
                "embeddings_df": "data/embedded_data.csv", 
                "corpus_df": "data/CleanedData_Augmented.csv",
                "bert_model": "/root/.cache/torch/NeMo/NeMo_1.22.0/bertlargeuncased/ca4ebba9f05a8ffb79845249ca046983/bertlargeuncased.nemo",            
                "demo": demo_folder,
            },
            signature=signature,
            pip_requirements=requirements
        )

 # 📜 Logging Model to MLflow

In [4]:
# Set the MLflow experiment name
mlflow.set_experiment(experiment_name="BERT Similarity Model")

# Start an MLflow run
with mlflow.start_run(run_name="BERT_Similarity_Run") as run:
    # Print the artifact URI for reference
    print(f"Run's Artifact URI: {run.info.artifact_uri}")
    
    # Log the BERT similarity model to MLflow
    BERTSimilarityModel.log_model(model_name="BERT_Similarity")

    # Register the logged model in MLflow Model Registry
    mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/BERT_Similarity", 
        name="BERT_Similarity"
    )

2025/03/12 17:25:49 INFO mlflow.tracking.fluent: Experiment with name 'BERT Similarity Model' does not exist. Creating a new experiment.


Run's Artifact URI: /phoenix/mlflow/667889996401907054/568df741a2b943c7b440da16232fa26d/artifacts


2025/03/12 17:25:54 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - transformers (current: 4.36.2, required: transformers==4.47.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Successfully registered model 'BERT_Similarity'.
Created version '1' of model 'BERT_Similarity'.


# 📦 Fetching the Latest Model Version from MLflow

In [5]:
# Initialize the MLflow client
client = MlflowClient()

# Retrieve the latest version of the "BERT_Similarity" model (not yet in a specific stage)
model_metadata = client.get_latest_versions("BERT_Similarity", stages=["None"])
latest_model_version = model_metadata[0].version  # Extract the latest model version

# Fetch model information, including its signature
model_info = mlflow.models.get_model_info(f"models:/BERT_Similarity/{latest_model_version}")

# Print the latest model version and its signature
print(f"Latest Model Version: {latest_model_version}")
print(f"Model Signature: {model_info.signature}")

[NeMo W 2025-03-12 17:26:29 nemo_logging:349] /tmp/ipykernel_1730/3842674751.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
      model_metadata = client.get_latest_versions("BERT_Similarity", stages=["None"])
    


Latest Model Version: 1
Model Signature: inputs: 
  ['query': string (required)]
outputs: 
  ['List of Pledges and Similarities': Tensor('object', (-1,))]
params: 
  ['show_score': boolean (default: False)]



# 🛠️ Loading the Model and Running Inference

In [6]:
# Load the trained BERT similarity model from MLflow
model = mlflow.pyfunc.load_model(model_uri=f"models:/BERT_Similarity/{latest_model_version}")

# Define a sample query for testing
query = "Give me a resort budget vacation suggestion"

# Use the model to predict similar results based on the query
result = model.predict({"query": [query]})

2025/03/12 17:26:29 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - transformers (current: 4.36.2, required: transformers==4.47.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
[NeMo W 2025-03-12 17:28:47 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    data_file: /home/yzhang/data/nlp/bert/47316/hdf5/lower_case_1_seq_len_512_max_pred_80_masked_lm_prob_0.15_random_seed_12345_dupe_factor_5_shard_1472_test_split_10/books_wiki_en_corpus/training/
    max_predictions_per_seq: 80
    batch_size: 16
    shuffle: true
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
    
[NeMo W 2025-0

[NeMo I 2025-03-12 17:28:50 modelPT:728] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 4.375e-05
        maximize: False
        weight_decay: 0.01
    )


[NeMo W 2025-03-12 17:28:50 lr_scheduler:890] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2025-03-12 17:28:53 save_restore_connector:249] Model BERTLMModel was successfully restored from /phoenix/mlflow/667889996401907054/568df741a2b943c7b440da16232fa26d/artifacts/BERT_Similarity/artifacts/bertlargeuncased.nemo.


# 📜 Displaying Results for the Input Query

In [7]:
# Convert the result into a pandas DataFrame
df = pd.DataFrame(result)

# Drop unnecessary columns if needed
df = df.drop(columns=["Unnamed: 0", "Topic"], errors="ignore")

# Rename columns for better readability
df.rename(columns={"Pledge": "Recommended Option", "Similarity": "Relevance Score"}, inplace=True)

# Display the DataFrame in a tabular format
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

╒════╤═════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════╕
│    │ Recommended Option                                                                                  │   Relevance Score │
╞════╪═════════════════════════════════════════════════════════════════════════════════════════════════════╪═══════════════════╡
│  0 │ For a budget-friendly vacation, consider a resort with vacation options and cruise activities.      │          0.869171 │
├────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────┤
│  1 │ For a budget-friendly vacation, consider a getaway with beach options and vacation activities.      │          0.863892 │
├────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────┤
│  2 │ For a budget-friendly vacation, consider a getaway with hotel options and vacation activit